In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import random_split, DataLoader, TensorDataset

class formatter:
    purple = '\033[95m'
    cyan = '\033[96m'
    darkcyan = '\033[36m'
    blue = '\033[94m'
    green = '\033[92m'
    yellow = '\033[93m'
    red = '\033[91m'
    bold = '\033[1m'
    underline = '\033[4m'
    reset = '\033[0m'

In [2]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [3]:
def display_info(data):
    print(formatter.bold + data + formatter.reset)
    data = eval(data)
    display(data.head())
    display(data.info())
    display(data.describe())
    print(f'{formatter.bold} Количество дубликатов: {formatter.reset}{data.duplicated().sum()}')

In [4]:
display_info('train_df')

train_df


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


 Количество дубликатов: 0


In [5]:
display_info('test_df')

test_df


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


None

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


 Количество дубликатов: 0


In [6]:
x_train = train_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']].copy()
y_train = train_df['Survived'].copy()

x_test = test_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']].copy()

In [7]:
x_train.isna().mean()

Pclass      0.000000
Sex         0.000000
Age         0.198653
SibSp       0.000000
Parch       0.000000
Fare        0.000000
Embarked    0.002245
dtype: float64

In [8]:
x_train['Embarked'].fillna('S', inplace=True)
x_train['Age'].fillna(x_train['Age'].median(), inplace=True)

In [9]:
x_test.isna().mean()

Pclass      0.000000
Sex         0.000000
Age         0.205742
SibSp       0.000000
Parch       0.000000
Fare        0.002392
Embarked    0.000000
dtype: float64

In [10]:
x_test['Fare'].fillna(x_test['Fare'].median(), inplace=True)
x_test['Age'].fillna(x_test['Age'].median(), inplace=True)

In [11]:
x_train_prepaired = x_train.copy()
y_train_prepaired = y_train.copy()
x_test_prepaired = x_test.copy()

In [12]:
def onehot_code(df, columns):
    df = df.copy()
    
    for column in columns:
        dummies = pd.get_dummies(df[column], prefix=column+'_')
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    
    return df

In [13]:
x_train = onehot_code(x_train, ['Pclass', 'Sex', 'Embarked'])
x_test = onehot_code(x_test, ['Pclass', 'Sex', 'Embarked'])

In [14]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [15]:
train_features = torch.tensor(x_train.astype(np.float32))
train_targets = torch.tensor(y_train.values.astype(np.int64))
test_features = torch.tensor(x_test.astype(np.float32))

train_dataset = TensorDataset(train_features, train_targets)
test_dataset = TensorDataset(test_features)

train_len = int(len(train_dataset) * 0.8)
val_len = len(train_dataset) - train_len
train, val = random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train, batch_size=32)
val_loader = DataLoader(val, batch_size=32)
test_loader = DataLoader(test_features, batch_size=1)

In [16]:
class Titanic_net(nn.Module):
    def __init__(self):
        super(Titanic_net, self).__init__()
        
        self.layer1 = nn.Linear(12, 32)
        self.dropout = nn.Dropout(0.3)
        self.layer2 = nn.Linear(32, 32)
        self.layer3 = nn.Linear(32, 2)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.layer1(x)))
        x = self.dropout(F.relu(self.layer2(x)))
        x = F.softmax(self.layer3(x))
        
        return x
    
model = Titanic_net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [17]:
for epoch in range(1, 30):
    print('Epoch:', epoch, end=' ')
    
    model.train()
    losses = []
    num_correct = 0
    num_samples = 0
    
    for x, y in train_loader:
        b = x.size(0)
        x = x.view(b, -1)
        
        preds = model(x)
#         print(preds, '\n\n', y)
        loss = criterion(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        _, predictions = preds.max(1)
        num_correct += (predictions == y).sum()
        num_samples += b
        
    print(f'Train loss: {torch.tensor(losses).mean():.2f}', end=' ')
    print(f'Train acc: {num_correct / num_samples * 100:.2f}%', end=' ')
    
    model.eval()
    losses = []
    num_correct = 0
    num_samples = 0
    
    for x, y in val_loader:
        b = x.size(0)
        x = x.view(b, -1)
        
        with torch.no_grad():
            preds = model(x)
        loss = criterion(preds, y)
        
        losses.append(loss.item())
        _, predictions = preds.max(1)
        num_correct += (predictions == y).sum()
        num_samples += b
        
    print(f'Validation loss: {torch.tensor(losses).mean():.2f}', end=' ')
    print(f'Validation acc: {num_correct / num_samples * 100:.2f}%')

Epoch: 1 Train loss: 0.56 Train acc: 73.60% Validation loss: 0.52 Validation acc: 77.65%
Epoch: 2 Train loss: 0.49 Train acc: 81.46% Validation loss: 0.50 Validation acc: 79.89%
Epoch: 3 Train loss: 0.48 Train acc: 82.87% Validation loss: 0.50 Validation acc: 80.45%
Epoch: 4 Train loss: 0.49 Train acc: 82.30% Validation loss: 0.50 Validation acc: 80.45%
Epoch: 5 Train loss: 0.48 Train acc: 82.58% Validation loss: 0.50 Validation acc: 82.12%
Epoch: 6 Train loss: 0.48 Train acc: 83.57% Validation loss: 0.50 Validation acc: 80.45%
Epoch: 7 Train loss: 0.48 Train acc: 82.87% Validation loss: 0.50 Validation acc: 81.01%
Epoch: 8 

<ipython-input-16-b56fabbebe13>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.layer3(x))


Train loss: 0.48 Train acc: 82.72% Validation loss: 0.50 Validation acc: 81.01%
Epoch: 9 Train loss: 0.48 Train acc: 83.01% Validation loss: 0.50 Validation acc: 81.01%
Epoch: 10 Train loss: 0.47 Train acc: 83.57% Validation loss: 0.50 Validation acc: 81.01%
Epoch: 11 Train loss: 0.48 Train acc: 82.72% Validation loss: 0.50 Validation acc: 81.01%
Epoch: 12 Train loss: 0.49 Train acc: 82.30% Validation loss: 0.50 Validation acc: 80.45%
Epoch: 13 Train loss: 0.48 Train acc: 83.29% Validation loss: 0.50 Validation acc: 80.45%
Epoch: 14 Train loss: 0.48 Train acc: 83.57% Validation loss: 0.50 Validation acc: 81.56%
Epoch: 15 Train loss: 0.48 Train acc: 82.87% Validation loss: 0.49 Validation acc: 82.12%
Epoch: 16 Train loss: 0.47 Train acc: 83.71% Validation loss: 0.49 Validation acc: 82.68%
Epoch: 17 Train loss: 0.47 Train acc: 83.85% Validation loss: 0.49 Validation acc: 81.01%
Epoch: 18 Train loss: 0.47 Train acc: 84.13% Validation loss: 0.49 Validation acc: 82.12%
Epoch: 19 Train loss:

In [18]:
model.eval()
answers = []

for x in test_loader:
    b = x.size(0)
    x = x.view(b, -1)

    with torch.no_grad():
        preds = model(x)

    _, predictions = preds.max(1)
    answers.append(np.array(predictions)[0])

<ipython-input-16-b56fabbebe13>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.layer3(x))


In [19]:
answers_df = pd.DataFrame(data=zip(np.arange(892, 1310), answers), columns=['PassengerId','Survived'])
answers_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [20]:
answers_df.to_csv('./my_submission.csv', index=False)

In [27]:
import catboost
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

x_train, x_validation, y_train, y_validation = train_test_split(
    x_train_prepaired, y_train_prepaired, train_size=0.8
)

boosting_model = catboost.CatBoostClassifier(
    n_estimators=600, learning_rate=0.01, cat_features=['Pclass', 'Sex', 'Embarked'], logging_level='Silent')
boosting_model.fit(x_train, y_train)

y_train_pred = boosting_model.predict_proba(x_train)[:, 1]
y_validation_pred = boosting_model.predict_proba(x_validation)[:, 1]

train_auc = roc_auc_score(y_train, y_train_pred)
validation_auc = roc_auc_score(y_validation, y_validation_pred)

display(train_auc)
display(validation_auc)

0.9073299045313971

0.8890604890604891

In [28]:
submission = pd.read_csv('./my_submission.csv')
submission['Survived'] = boosting_model.predict(x_test_prepaired)
# submission['Survived'] = 1 - submission['Survived']
submission.to_csv('./my_submission.csv', index=False)